# Import libraries

In [30]:
import pandas as pd
import sqlalchemy
import math

# Function to split data frame

In [8]:
def split_csv(file_path,json:int=30,database:int=50,csv:int=20):
    df=pd.read_csv(file_path)
    #randomazie data
    df=df.sample(frac=1)

    #get the lengh of data to calculate the number of lines for each dataframe
    df_len=len(df.index)

    #check if the percentage of spliting under 100
    if((json+database+csv)!=100):
        raise Exception('The perecentage of spliting must be equal 100') 

    #Calculate the points of json, EX: [0,400] the json will containt all rows from index 0 to 400
    start_index=0
    end_index=int((df_len*json)/100)
    json_points=[start_index,end_index]

    #The csv file will start in the end of json file 
    start_index=end_index
    end_index=int((df_len*(json+csv))/100)
    csv_points=[start_index,end_index]

    #Databasefile
    start_index=end_index
    end_index=df_len
    database_points=[start_index,end_index]

    #return 3 dataframe Json, CSV, Database
    return df.iloc[json_points[0]:json_points[1]],df.iloc[database_points[0]:database_points[1]],df.iloc[csv_points[0]:csv_points[1]]

# Test function

In [36]:
try:
    json_per = math.floor(float(input('Taux de json: ')))
    database_per = math.floor(float(input('Taux de database: ')))
    csv_per = math.floor(float(input('Taux de csv: ')))

    #split data to 3 data frame
    df_json,df_database,df_csv=split_csv('dataset/real_estate_dataset.csv',json=json_per,database=database_per,csv=csv_per)
    df_json.to_json('exported_dataset/real_estate.json')
    df_csv.to_csv('exported_dataset/real_estate.csv')

    #insert to database
    server = 'ELMEHDI'
    database = 'realestate'
    driver = 'ODBC Driver 17 for SQL Server'
    connection_string = f'mssql+pyodbc://{server}/{database}?driver={driver}'
    # create a SQLAlchemy engine using the connection string
    engine = sqlalchemy.create_engine(connection_string)
    # write the DataFrame to a table in the database
    table_name = 'house'
    df_database.to_sql(table_name, engine, if_exists='replace', index=False)

    print('Done')
except Exception as e:
    print(e.args)


Done
